In [1]:
#IT SHOULD BE RUN ON KAGGLE


!pip install tensorflow

# Import necessary libraries
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GlobalMaxPooling1D
# For tokenization   # For stopword removal
!pip install contractions
import spacy
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import contractions

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    tf.config.experimental.enable_op_determinism()  # Enforce deterministic ops in TensorFlow

set_seed(42)  # Set a global seed


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.6 MB/s eta 0:00:00


In [28]:
# Load the dataset
#I have used relative path to load dataset, because in kaggle there is word count restriction while uploading the data, and in kaggle documentation they mentioned the method of keeping the training set in folder named dataset.
file_path = '../input/dataset/CW2-training-dataset.csv'  # Adjust the file path based on your Kaggle dataset
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print("Dataset Loaded:")
print(data.head())


Dataset Loaded:
                                     ID                        title  \
0  8f5203de-b2f8-4c0c-b0c1-835ba92422e9                   Si wang ta   
1  6416fe15-6f8a-41d4-8a78-3e8f120781c7          Shattered Vengeance   
2  4979fe9a-0518-41cc-b85f-f364c91053ca                 L'esorciccio   
3  b672850b-a1d9-44ed-9cff-025ee8b61e6f  Serendipity Through Seasons   
4  b4d8e8cc-a53e-48f8-be6a-6432b928a56d                The Liability   

                                       plot_synopsis  comedy  cult  flashback  \
0  After a recent amount of challenges, Billy Lo ...       0     0          0   
1  In the crime-ridden city of Tremont, renowned ...       0     0          0   
2  Lankester Merrin is a veteran Catholic priest ...       0     1          0   
3  "Serendipity Through Seasons" is a heartwarmin...       0     0          0   
4  Young and naive 19-year-old slacker, Adam (Jac...       0     0          1   

   historical  murder  revenge  romantic  scifi  violence  
0   

In [29]:
# Identify genre columns (assuming the last 9 columns are the genres)
genre_columns = ['comedy', 'cult', 'flashback', 'historical', 'murder', 'revenge', 'romantic', 'scifi', 'violence']

# Filter rows where all genre columns are 0
rows_before = data.shape[0]
data_filtered = data[data[genre_columns].sum(axis=1) > 0]
data_filtered = data_filtered.sample(frac=1, random_state=42).reset_index(drop=True)
data_filtered['text'] = data_filtered['title'] + ' ' + data_filtered['plot_synopsis']
rows_after = data_filtered.shape[0]

# Calculate the number of rows removed
removed_movies_count = rows_before - rows_after

# Display the result
print(f"Number of movies removed (no genres): {removed_movies_count}")
print(f"Number of movies remaining: {rows_after}")
lemmatizer = WordNetLemmatizer()

Number of movies removed (no genres): 0
Number of movies remaining: 8257


In [30]:
# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Define stopwords
stop_words = set(stopwords.words('english'))

def normalize_contractions(text):
    """
    Expands contractions in a given text.
    Example: "can't" -> "cannot"
    """
    return contractions.fix(text)

def preprocess_text(text):
    text = normalize_contractions(text)
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphanumeric characters
    doc = nlp(text)  # Process text with spaCy
    # Tokenize, remove stopwords, and lemmatize
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    return ' '.join(tokens)  # Join lemmatized


# Apply preprocessing to the 'plot_synopsis' column
data_filtered['cleaned_plot'] = data_filtered['plot_synopsis'].apply(preprocess_text)

# Tokenize and pad sequences
MAX_VOCAB_SIZE = 20000  # Define maximum vocabulary size
MAX_SEQUENCE_LENGTH = 300  # Define maximum sequence length for padding

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(data_filtered['cleaned_plot'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(data_filtered['cleaned_plot'])

# Pad sequences to ensure equal length
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Display a sample of the processed data
print("Sample of preprocessed text with spaCy lemmatization and padded sequence:")
print(data_filtered[['plot_synopsis', 'cleaned_plot']].head())
print("\nSample Padded Sequence:")
print(padded_sequences[:5])


Sample of preprocessed text with spaCy lemmatization and padded sequence:
                                       plot_synopsis  \
0  The film tells the story of Antonio Ricci, an ...   
1  In the near future, the Moon has been colonize...   
2  Two miners (Peter Hock & Harry Bellaver) find ...   
3  Set in 12th-century medieval Japan, the film t...   
4  Despite some deviations, the book's historical...   

                                        cleaned_plot  
0  film tell story antonio ricci unemployed man d...  
1  near future moon colonize support station surf...  
2  two miner peter hock   harry bellaver find yog...  
3  set thcentury medieval japan film take place w...  
4  despite deviation book historical framework ge...  

Sample Padded Sequence:
[[   72     3   113 ...  3670   656     0]
 [  400   612  1312 ...     0     0     0]
 [   16  4147   276 ...   161   302  1965]
 [  109 12557  8362 ...     0     0     0]
 [  263     1   316 ...  1726  9509 16073]]


In [31]:
# Load GloVe embeddings
def load_glove_embeddings(glove_file_path, embedding_dim):
    embeddings_index = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Path to GloVe embeddings (update with actual file path)
# I have used glove.6B.300d.txt
GLOVE_FILE_PATH = '../input/glove/glove.6B.300d.txt'
EMBEDDING_DIM = 300  # Using GloVe with 300 dimensions
glove_embeddings = load_glove_embeddings(GLOVE_FILE_PATH, EMBEDDING_DIM)

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Calculate class weights
genre_counts = data_filtered[genre_columns].sum()
total_samples = len(data_filtered)
class_weights = {i: total_samples / (len(genre_columns) * count) for i, count in enumerate(genre_counts)}


model = Sequential([
    Embedding(input_dim=vocab_size, 
              output_dim=EMBEDDING_DIM, 
              input_length=MAX_SEQUENCE_LENGTH, 
              weights=[embedding_matrix], 
              trainable=False),
    Bidirectional(LSTM(128, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='sigmoid')
])


early_stopping = EarlyStopping(
    monitor='loss', patience=2, restore_best_weights=True
)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0005),  # Adjust learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    padded_sequences,
    data_filtered[genre_columns].values,
    epochs=20,
    batch_size=64,  # Increased batch size for faster training
    verbose=1,
    class_weight=class_weights,
    callbacks=[early_stopping]
  # Handle class imbalance
)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.1599 - loss: 0.3197
Epoch 2/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.1978 - loss: 0.2837
Epoch 3/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.2405 - loss: 0.2683
Epoch 4/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.2612 - loss: 0.2576
Epoch 5/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.2580 - loss: 0.2530
Epoch 6/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.2598 - loss: 0.2482
Epoch 7/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.2709 - loss: 0.2430
Epoch 8/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.2774 - loss: 0.2377
Epoch 9/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.2917 - loss: 0.2319
Epoch 10/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.2865 - loss: 0.2304
Epoch 11/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.2904 - loss: 0.2233
Epoch 12/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 

In [32]:
 #Preprocess and Predict
def preprocess_and_predict(input_file_path, output_file_path, tokenizer, model, max_sequence_length, genre_columns, threshold=0.3):
    """
    Preprocess the input dataset and make predictions using the trained model.
    Dynamically handles datasets with varying structures (e.g., validation or test datasets).
    """
    data = pd.read_csv(input_file_path)
    text_column = 'plot_synopsis' if 'plot_synopsis' in data.columns else data.columns[-1]
    id_column = 'ID' if 'ID' in data.columns else data.columns[0]

    data['cleaned_plot'] = data[text_column].apply(preprocess_text)
    sequences = tokenizer.texts_to_sequences(data['cleaned_plot'])
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

    predictions = model.predict(padded_sequences, batch_size=32)
    binary_predictions = (predictions > threshold).astype(int)
    for i in range(len(binary_predictions)):
        if binary_predictions[i].sum() == 0:
            top_genres = predictions[i].argsort()[-1:]
            binary_predictions[i][top_genres] = 1

    output_df = pd.DataFrame(binary_predictions, columns=genre_columns)
    output_df.insert(0, 'ID', data[id_column])
    output_df.to_csv(output_file_path, index=False)
    print(f"Predictions saved to: {output_file_path}")

# Example Usage for Test Data
test_file_path = '../input/dataset/CW2-test-dataset.csv'
output_test_path = '../working/11028972_task1_results.csv'

preprocess_and_predict(
    input_file_path=test_file_path,
    output_file_path=output_test_path,
    tokenizer=tokenizer,
    model=model,
    max_sequence_length=MAX_SEQUENCE_LENGTH,
    genre_columns=genre_columns
)



38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Predictions saved to: ../working/11028972_task1_results.csv
